In [ ]:
import sys
import os
# Add the parent directory to the path to find custom modules
sys.path.append(os.path.abspath('..'))

from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

from aerodynamics import calculate_lift_drag, check_ground_effect
from analysis import plot_complexity_dashboard
from boundaries import TunnelBoundaries
from lbm_core import LBMSolver

In [ ]:

# --- CONFIGURATION ---
NX, NY = 900, 200
REYNOLDS = 5_000        # High speed turbulence
FRAMES = 1200           # Total video frames (1000 physics steps at 5/frame)
STEPS_PER_FRAME = 50    # ← Physics iterations per animation frame
U_INLET = 0.08          # Realistic F1 wind tunnel speed
CS_SMAG = 0.17          # Smagorinsky constant for stability

# Create timestamped output folder
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = f"run_{timestamp}_output"
os.makedirs(output_folder, exist_ok=True)
print(f"Output folder: {output_folder}")

# --- SETUP SIMULATION ---
print("Initializing Simulation for Video Rendering...")
solver = LBMSolver(NX, NY, REYNOLDS, u_inlet=U_INLET)
bounds = TunnelBoundaries(NX, NY)

ride_height = 16

bounds.add_ground(type="no_slip")
bounds.add_f1_wing_proxy(x_pos=150, height=ride_height, length=60, slope=0.45)
bounds.add_rectangular_obstacle(x_start=210, y_start=ride_height + 1, length=120, height=20)
bounds.add_reverse_triangle(x_pos=240, height=ride_height + 20, length=90, slope=2/9)
bounds.add_f1_wing_proxy(x_pos=300, height=ride_height + 30, length=30, slope=0.2)

relevant_x_start = 145
relevant_x_end = 335
relevant_y_start = 5
relevant_y_end = 75

# --- SETUP PLOTS ---
fig = plt.figure(figsize=(16, 12), dpi=150)
gs = fig.add_gridspec(3, 1, height_ratios=[2, 2, 1])

# Plot 1: Velocity Magnitude (Top)
ax_flow = fig.add_subplot(gs[0])
velocity_mag = np.zeros((NY, NX))
velocity_mag[bounds.mask] = np.nan
im_flow = ax_flow.imshow(velocity_mag, origin='lower', cmap='magma', vmin=0, vmax=0.15)
ax_flow.set_title(f"Velocity Magnitude (Re={REYNOLDS})")
plt.colorbar(im_flow, ax=ax_flow, label="Speed |u|", orientation="horizontal", pad=0.1)

# Plot 2: Streamlines (Middle)
ax_stream = fig.add_subplot(gs[1])
y_coords, x_coords = np.mgrid[0:NY:15, 0:NX:15]

# Data containers
lift_data = []
drag_data = []
frame_indices = []

print("Starting Render...")
for frame in range(FRAMES):
    # Run physics steps
    for _ in range(STEPS_PER_FRAME):
        solver.collide_and_stream(bounds.mask)
        bounds.apply_inlet_outlet(solver)
    
    # Update velocity magnitude
    v_mag = np.sqrt(solver.u[:,:,0]**2 + solver.u[:,:,1]**2)
    v_mag[bounds.mask] = np.nan
    im_flow.set_array(v_mag)
    
    # Update streamlines
    ax_stream.clear()
    u_sample = solver.u[::15, ::15, 0]
    v_sample = solver.u[::15, ::15, 1]
    speed_sample = np.sqrt(u_sample**2 + v_sample**2)
    ax_stream.streamplot(x_coords, y_coords, u_sample, v_sample, 
                         color=speed_sample, cmap='plasma', density=1.5, 
                         linewidth=1, arrowsize=1.2)
    ax_stream.set_title("Flow Streamlines")
    ax_stream.set_xlabel("X")
    ax_stream.set_ylabel("Y")
    ax_stream.set_xlim(0, NX)
    ax_stream.set_ylim(0, NY)
    ax_stream.set_aspect('equal')
        
    # Save frame
    frame_filename = os.path.join(output_folder, f"frame_{frame:04d}.png")
    plt.savefig(frame_filename, dpi=150, bbox_inches='tight')
    
    if frame % 10 == 0:
        print(f"Rendered frame {frame}/{FRAMES}")

plt.close(fig)
print(f"\n✓ Rendering complete! {FRAMES} frames saved to '{output_folder}/'")
print(f"\nTo create video with ffmpeg:")
print(f"  cd {output_folder}")
print(f"  ffmpeg -framerate 18 -pattern_type glob -i '*.png' -c:v libx264 -pix_fmt yuv420p output.mp4")


In [22]:
# --- COMPREHENSIVE VISUALIZATION: 4-Panel Display ---
print("Creating comprehensive multi-panel visualization...")

from aerodynamics import BoundaryForceCalculator
from datetime import datetime

# Create separate output folder with new timestamp
comp_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
comp_output_folder = f"run_{comp_timestamp}_comprehensive"
os.makedirs(comp_output_folder, exist_ok=True)
print(f"Comprehensive output folder: {comp_output_folder}")

# Reset solver for fresh run
solver = LBMSolver(NX, NY, REYNOLDS, u_inlet=U_INLET)

# Initialize force calculator
force_calc = BoundaryForceCalculator(bounds, solver, 
                                     x_start=relevant_x_start, x_end=relevant_x_end,
                                     y_start=relevant_y_start, y_end=relevant_y_end)

# Setup figure with 2x2 grid (16:9 aspect ratio)
fig = plt.figure(figsize=(19.2, 10.8), dpi=150)
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.25)

# Panel 1: Velocity Magnitude (Top Left) - JET COLORMAP
ax1 = fig.add_subplot(gs[0, 0])
velocity_mag = np.zeros((NY, NX))
velocity_mag[bounds.mask] = np.nan
im1 = ax1.imshow(velocity_mag, origin='lower', cmap='jet', vmin=0, vmax=0.15, aspect=16/9)
ax1.set_title('Velocity Magnitude')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
plt.colorbar(im1, ax=ax1, label='|u|', orientation='horizontal', pad=0.1)

# Panel 2: Pressure Distribution (Top Right)
ax2 = fig.add_subplot(gs[0, 1])
pressure = solver.rho / 3.0
pressure_masked = pressure.copy()
pressure_masked[bounds.mask] = np.nan
im2 = ax2.imshow(pressure_masked, origin='lower', cmap='RdBu_r', aspect=16/9)
ax2.set_title('Pressure Distribution')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
cbar2 = plt.colorbar(im2, ax=ax2, label='Pressure', orientation='horizontal', pad=0.1)

# Panel 3: Streamlines (Bottom Left)
ax3 = fig.add_subplot(gs[1, 0])
y_coords_stream, x_coords_stream = np.mgrid[0:NY:15, 0:NX:15]
ax3.set_title('Flow Streamlines')
ax3.set_xlabel('X')
ax3.set_ylabel('Y')
ax3.set_xlim(0, NX)
ax3.set_ylim(0, NY)
ax3.set_aspect(16/9)

# Panel 4: Force History (Bottom Right)
ax4 = fig.add_subplot(gs[1, 1])
drag_history = []
lift_history = []
step_history = []
line_drag, = ax4.plot([], [], 'r-', linewidth=2, label='Drag (F_x)')
line_lift, = ax4.plot([], [], 'b-', linewidth=2, label='Lift (F_y)')
ax4.axhline(0, color='k', linestyle='--', alpha=0.3)
ax4.set_xlabel('Simulation Step')
ax4.set_ylabel('Force')
ax4.set_title('Aerodynamic Forces')
ax4.legend(loc='upper right')
ax4.grid(True, alpha=0.3)
ax4.set_xlim(0, FRAMES * STEPS_PER_FRAME)
ax4.set_ylim(-2, 2)

print("Starting comprehensive render...")
for frame in range(FRAMES):
    # Run physics
    for _ in range(STEPS_PER_FRAME):
        solver.collide_and_stream(bounds.mask)
        bounds.apply_inlet_outlet(solver)
    
    # Calculate forces
    fx, fy = force_calc.calculate()
    drag_history.append(fx)
    lift_history.append(fy)
    step_history.append(frame * STEPS_PER_FRAME)
    
    # Update Panel 1: Velocity Magnitude
    v_mag = np.sqrt(solver.u[:,:,0]**2 + solver.u[:,:,1]**2)
    v_mag[bounds.mask] = np.nan
    im1.set_array(v_mag)
    
    # Update Panel 2: Pressure
    pressure = solver.rho / 3.0
    pressure_masked = pressure.copy()
    pressure_masked[bounds.mask] = np.nan
    p_mean = np.nanmean(pressure_masked)
    p_std = np.nanstd(pressure_masked)
    im2.set_array(pressure_masked)
    im2.set_clim(p_mean - 2*p_std, p_mean + 2*p_std)
    
    # Update Panel 3: Streamlines
    ax3.clear()
    u_sample = solver.u[::15, ::15, 0]
    v_sample = solver.u[::15, ::15, 1]
    speed_sample = np.sqrt(u_sample**2 + v_sample**2)
    ax3.streamplot(x_coords_stream, y_coords_stream, u_sample, v_sample, 
                   color=speed_sample, cmap='plasma', density=1.5, 
                   linewidth=1, arrowsize=1.2)
    ax3.set_title('Flow Streamlines')
    ax3.set_xlabel('X')
    ax3.set_ylabel('Y')
    ax3.set_xlim(0, NX)
    ax3.set_ylim(0, NY)
    ax3.set_aspect(16/9)
    
    # Update Panel 4: Force History
    line_drag.set_data(step_history, drag_history)
    line_lift.set_data(step_history, lift_history)
    
    # Save frame
    frame_filename = os.path.join(comp_output_folder, f"comprehensive_{frame:04d}.png")
    plt.savefig(frame_filename, dpi=150, bbox_inches='tight')
    
    if frame % 10 == 0:
        print(f"Frame {frame}/{FRAMES} - Step {frame*STEPS_PER_FRAME} - Drag: {fx:.4f}, Lift: {fy:.4f}")

plt.close(fig)
print(f"\n✓ Comprehensive visualization complete!")
print(f"Output: {comp_output_folder}/comprehensive_XXXX.png")
print(f"\nCreate video with:")
print(f"  cd {comp_output_folder}")
print(f"  ffmpeg -framerate 18 -i comprehensive_%04d.png -c:v libx264 -pix_fmt yuv420p comprehensive.mp4")

Creating comprehensive multi-panel visualization...
Comprehensive output folder: run_20260130_002427_comprehensive
Starting comprehensive render...
Frame 0/1200 - Step 0 - Drag: -11.6083, Lift: 1.3868
Frame 10/1200 - Step 500 - Drag: -5.0586, Lift: -0.2890
Frame 20/1200 - Step 1000 - Drag: -4.7279, Lift: 0.3542
Frame 30/1200 - Step 1500 - Drag: -4.0727, Lift: -0.9756
Frame 40/1200 - Step 2000 - Drag: -2.9056, Lift: 0.2692
Frame 50/1200 - Step 2500 - Drag: -3.5107, Lift: -0.5559
Frame 60/1200 - Step 3000 - Drag: -2.7892, Lift: -0.9195
Frame 70/1200 - Step 3500 - Drag: -2.3959, Lift: -0.5451
Frame 80/1200 - Step 4000 - Drag: -2.4715, Lift: -0.9699
Frame 90/1200 - Step 4500 - Drag: -2.9512, Lift: 0.3926
Frame 100/1200 - Step 5000 - Drag: -2.9110, Lift: -0.8352
Frame 110/1200 - Step 5500 - Drag: -2.7158, Lift: -0.8843
Frame 120/1200 - Step 6000 - Drag: -2.5038, Lift: -0.7052
Frame 130/1200 - Step 6500 - Drag: -2.0010, Lift: -1.3189
Frame 140/1200 - Step 7000 - Drag: -2.4921, Lift: -0.7031
